In [65]:
import tensorflow as tf
from tensorflow.keras.datasets import boston_housing
import numpy as np
from tensorflow.keras import models
from tensorflow.keras import layers

In [66]:
(train_data, train_targets), (test_data, test_targets)=boston_housing.load_data()

In [67]:
print(train_data[0])
print(train_targets[0])

[  1.23247   0.        8.14      0.        0.538     6.142    91.7
   3.9769    4.      307.       21.      396.9      18.72   ]
15.2


In [74]:
print("shape of train_data:",train_data.shape )
print("shape of test_data:",test_data.shape )


shape of train_data: (404, 13)
shape of test_data: (102, 13)


In [69]:
mean = train_data.mean(axis=0)
print("mean:", mean)
train_data -= mean
print("train_data:", train_data)
std = train_data.std(axis=0)
print("std:", std)
train_data /= std
print("train_data:", train_data)
test_data -= mean
test_data /= std

mean: [3.74511057e+00 1.14801980e+01 1.11044307e+01 6.18811881e-02
 5.57355941e-01 6.26708168e+00 6.90106436e+01 3.74027079e+00
 9.44059406e+00 4.05898515e+02 1.84759901e+01 3.54783168e+02
 1.27408168e+01]
train_data: [[ -2.51264057 -11.48019802  -2.96443069 ...   2.5240099   42.11683168
    5.97918317]
 [ -3.72334057  71.01980198  -9.07443069 ...  -3.7759901   40.59683168
   -9.63081683]
 [  1.15310943 -11.48019802   6.99556931 ...   1.7240099   20.73683168
   -9.48081683]
 ...
 [ -3.71045057  23.51980198  -5.04443069 ...  -1.5759901    7.46683168
   -4.91081683]
 [ -1.59593057 -11.48019802   8.47556931 ...  -3.7759901  -92.83316832
    3.04918317]
 [ -3.73072057  48.51980198  -8.17443069 ...  -2.8759901   21.91683168
   -8.36081683]]
std: [9.22929073e+00 2.37382770e+01 6.80287253e+00 2.40939633e-01
 1.17147847e-01 7.08908627e-01 2.79060634e+01 2.02770050e+00
 8.68758849e+00 1.66168506e+02 2.19765689e+00 9.39946015e+01
 7.24556085e+00]
train_data: [[-0.27224633 -0.48361547 -0.43576161

In [70]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu',
                            input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model



In [71]:
k = 4
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []

for i in range(k):
    print('processing fold #', i)
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
        train_data[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
        train_targets[(i + 1) * num_val_samples:]],
        axis=0)
    model = build_model()
    model.fit(partial_train_data, partial_train_targets,
            epochs=num_epochs, batch_size=1, verbose=0)
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3
